In [2]:
import re

from bs4 import BeautifulSoup
import pandas as pd
import PyPDF2
import requests
import wget
import urllib.request

ImportError: dlopen(//anaconda/lib/python3.5/site-packages/numpy/core/multiarray.cpython-35m-darwin.so, 10): Library not loaded: @loader_path/../../../../libmkl_intel_lp64.dylib
  Referenced from: //anaconda/lib/python3.5/site-packages/numpy/core/multiarray.cpython-35m-darwin.so
  Reason: image not found

In [ ]:
r = requests.get('https://www.colorado.gov/pacific/revenue/colorado-marijuana-tax-data')
soup = BeautifulSoup(r.text, "lxml")

In [ ]:
anchor_tags = soup.findAll('div','fieldset-wrapper')[1].findAll('a')

In [ ]:
links = [tag['href'] for tag in anchor_tags]

In [ ]:
test_link = links[0]

In [ ]:
#wget needs percent encoding to go away
wget.download(urllib.request.unquote(test_link))

In [ ]:
with open('0115 Marijuana Sales Tax by County Report.pdf','rb') as file:
    pdfReader = PyPDF2.PdfFileReader(file)
    text = pdfReader.getPage(0).extractText()

In [ ]:
text = text.replace('\n',' ')

In [ ]:
irrelevant_text_loc = text.find('Remainder of State is comprised of the following counties')

In [ ]:
text = text[:irrelevant_text_loc]

In [ ]:
text

In [ ]:
counties = ['Adams','Arapahoe','Boulder','Clear Creek','Denver']

In [ ]:
county_revenues = []

for county in counties:
    regex = re.compile(r'{county}\s[\$,0-9\s]+'.format(county=county))
    county_revenues.append(regex.findall(text))

In [ ]:
df = pd.DataFrame()

for revenue in county_revenues:
    for table in revenue:
        county = re.compile('[a-zA-Z]+\s?[a-zA-Z]?').findall(table)
        revenues = re.compile(r'[\$,0-9]+').findall(table)
        #Medical marijuana table has one value
        if len(revenues)==1: 
            df.loc[county,'Medical Marijuana'] = revenues[0]
        #Retail table has two values
        elif len(revenues)==2:
            df.loc[county,'Retail Sales Tax'] = revenues[0]
            df.loc[county,'Additional Retail Sales Tax'] = revenues[1]